# IMC-DIMR and marker-individual exports

Hot pixel removal by removing local maxima in adjacent 3x3 sliding windows and export by marker

In [2]:
import sys
sys.path.append("/home/daria_l/src/MapMet/code/python/Image_Preprocessing/IMC-Denoise_adapted")

In [3]:
import IMC_Denoise
from IMC_Denoise.IMC_Denoise_main.DIMR import DIMR
import pandas as pd
import os, glob
import tifffile

After spillover compensation in R, we first have to convert the images to float32 as they are written from R in uint32. And we also have to attach channel names to the ome-tiffs.

In [3]:
import os
import tifffile
import pandas as pd
import glob
from xtiff import to_tiff
import numpy as np

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
samples = os.listdir(os.path.join(base_path, "BM"))

out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20221220_Cellpose_IFbased")
marker_codes = pd.read_csv(os.path.join(out_path, 'marker_codes.csv'), sep=";")

for s in samples:
    mc_image_files = glob.glob(os.path.join(out_path + "/comp_img/{}*.tiff".format(s)), recursive=True)

    if not os.path.exists(os.path.join(base_path, "BM", s, "IF_IMC_comp_196")):
        os.mkdir(os.path.join(base_path, "BM", s, "IF_IMC_comp_196"))

    for f in mc_image_files:
        img = tifffile.imread(f)
        img= img.astype(np.float32)

        to_tiff(
            img,
            os.path.join(base_path, "BM", s, "IF_IMC_comp_196", f.split("/")[-1]),
            image_name=os.path.join(base_path, "BM", s, "IF_IMC_comp_196", f.split("/")[-1]),
            channel_names=list(marker_codes["new_names"]),
            pixel_size=1.0,
            pixel_depth=1.0,
            )

/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220106_16-1454_BM_ROI_002_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220106_16-1454_BM_ROI_001_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220106_16-1454_BM_ROI_004_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does not end with .ome.tiff: 20220106_16-1454_BM_ROI_003_IF_IMC.tiff
  warnings.warn(
/home/daria_l/anaconda3/envs/mapmet/lib/python3.10/site-packages/xtiff/tiff.py:130: UserWarning: The specified OME-TIFF file name does n

Delete below part later. This is just for when the training samples are already randomly selected.

In [8]:
# import os
# import tifffile
# import xml.etree.ElementTree
# import pandas as pd
# import io
# import random
# import glob
# import numpy as np

# def read_channel_names(path):
    
#     tiff = tifffile.TiffFile(path)
#     omexml_string = tiff.pages[0].description
#     root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
#     namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
#     channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
#     channel_names = [c.attrib['Name'] for c in channels]
#     return channel_names


# base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

# BM_path = os.path.join(base_path, "BM")
# out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")

# training_samples = os.listdir(os.path.join(out_path, "training", "raw"))

# for s in training_samples: 

#     sample = s.split("_ROI")[0]
#     training_file = s.split(".")[0]

#     multichannel_path = os.path.join(BM_path, sample, "IF_IMC_comp_196")

#     mc_image_files = os.listdir(multichannel_path)

#     for f in mc_image_files:
#         file_name = f.split(".")[0]

#         mc_image = tifffile.imread(os.path.join(multichannel_path, f))

#         channel_names = read_channel_names(os.path.join(multichannel_path, f))

#         if file_name == training_file:
#             folder = "training"
#         else:
#             folder = "other"
        
#         if not os.path.exists(os.path.join(out_path, folder, "comp_196", file_name)):
#             os.mkdir(os.path.join(out_path, folder, "comp_196", file_name))

#         for c, img in enumerate(mc_image):
#             tifffile.imwrite(os.path.join(out_path, folder, "comp_196", file_name, channel_names[c]+".tiff"),img)


After spillover compensation, we have to prepare the file structure that is required by IMC-Denoise for DIMR hot-pixel removal.

In [ ]:
import os
import tifffile
import xml.etree.ElementTree
import pandas as pd
import io
import random
import glob
import numpy as np

def read_channel_names(path):
    
    tiff = tifffile.TiffFile(path)
    omexml_string = tiff.pages[0].description
    root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
    namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
    channel_names = [c.attrib['Name'] for c in channels]
    return channel_names


base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

BM_path = os.path.join(base_path, "BM")
out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")

if not os.path.exists(os.path.join(out_path, "training")):
    os.mkdir(os.path.join(out_path, "training"))

if not os.path.exists(os.path.join(out_path, "other")):
    os.mkdir(os.path.join(out_path, "other"))

samples = os.listdir(BM_path)

for s in samples: 

    multichannel_path = os.path.join(BM_path, s, "IF_IMC_comp_196")

    mc_image_files = os.listdir(multichannel_path)

    training_file = random.choice(mc_image_files)
    training_file = training_file.split(".")[0]


    for f in mc_image_files:
        file_name = f.split(".")[0]

        mc_image = tifffile.imread(os.path.join(multichannel_path, f))

        channel_names = read_channel_names(os.path.join(multichannel_path, f))

        if file_name == training_file:
            folder = "training"
        else:
            folder = "other"

        if not os.path.exists(os.path.join(out_path, folder, "comp_196", file_name)):
            os.mkdir(os.path.join(out_path, folder, "comp_196", file_name))

        for c, img in enumerate(mc_image):
            tifffile.imwrite(os.path.join(out_path, folder, "comp_196", file_name, channel_names[c]+".tiff"),img)


After we have generated single images out of the stacks for each RoI of each sample, we can apply DIMR hot-pixel removal. We will randomly select one RoI per sample for training of DIMR and Ilastik background prediction. 

In [ ]:
base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik"


marker_codes = pd.read_csv(os.path.join(base_path, 'marker_codes.csv'), sep=";")

n_neighbours = 4 # Larger n enables removing more consecutive hot pixels. 
n_iter = 3 # Iteration number for DIMR
window_size = 3 # Slide window size. For IMC images, window_size = 3 is fine.

other_path = os.path.join(base_path, "other")
training_path = os.path.join(base_path, "training")

for m in marker_codes["new_names"]:

    if m in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]:
        continue

    working_path = training_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    # if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
    #     os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp_196/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)


    working_path = other_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    # if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
    #     os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp_196/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)
    

After DIMR correction, we will perform background prediciton in Ilastik and save the inferred masks. In a next step, we will go through all samples and per marker read in the comp-DIMR corrected images and ilastik masks of all RoI. If the marker is an IF marker or CD274, we will set the entire mask to one, since we did not perform fg/bg prediction for those markers.

In [ ]:
import os
import glob
import pandas as pd
from functools import reduce
import numpy as np 
import tifffile
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from xtiff import to_tiff
import re

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
BM_path = os.path.join(base_path, "BM")
samples = os.listdir(BM_path)

path_to_data = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")
marker_codes = pd.read_csv(os.path.join(path_to_data, 'marker_codes.csv'), sep=";")
marker_codes = [i for i in list(marker_codes.new_names) if i not in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]]

def merge_on_marker(folder:str, sample: str, data_path:str) -> pd.DataFrame:

    roi = os.listdir(os.path.join(BM_path, sample, "IF_IMC_lowres"))
    df_list = []

    for _r in roi:
        r = _r.split("_IF")[0]
        paths = glob.glob(os.path.join(data_path + "/*/" + folder + "/*/{}*.tiff".format(r)), recursive=True)
        marker = [i.split("/")[-2] for i in paths]
        df = pd.DataFrame({"marker": marker, r: paths})
        df_list.append(df)

    merged_df = reduce(lambda df1,df2: pd.merge(df1,df2,on="marker", how="outer"), df_list)
    merged_df.set_index("marker", inplace=True)

    return merged_df

def stack_save(df: pd.DataFrame, out_path: str):
    for _roi in df.columns:
        mc_image = np.stack(df[_roi])
        ch_names = df.index

        to_tiff(
            mc_image,
            os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            image_name=os.path.join(out_path, _roi + "_IF_IMC.tiff"),
            channel_names=list(ch_names),
            pixel_size=1.0,
            pixel_depth=1.0,
            )


for s in samples: 
    if not os.path.exists(os.path.join(BM_path, s, "IF_IMC_processed")):
        os.mkdir(os.path.join(BM_path, s, "IF_IMC_processed"))

    if not os.path.exists(os.path.join(BM_path, s, "IF_IMC_ilastik-masks")):
        os.mkdir(os.path.join(BM_path, s, "IF_IMC_ilastik-masks"))
    
    img_paths_df = merge_on_marker("DIMR", s, path_to_data)
    mask_paths_df = merge_on_marker("DIMR_Ilastik", s, path_to_data)

    mask_paths_df = mask_paths_df.loc[:, img_paths_df.columns]

    img_df = img_paths_df.loc[marker_codes, :]
    mask_df = img_paths_df.loc[marker_codes, :]

    for m in marker_codes:
        img_paths = img_paths_df.loc[m]
        imgs = np.array([tifffile.imread(f) for f in img_paths])

        if m in ["1IF_DAPI_IF1", "2IF_GD2-IF2", "3IF_CD56-IF3", "160Gd-CD274_Gd160"]:
            masks = np.ones_like(imgs)
        else:
            mask_paths = mask_paths_df.loc[m]
            masks = np.array([tifffile.imread(f) for f in mask_paths])


        imgs_cut = imgs[:, 2:-2, 2:-2]
        masks_cut = masks[:, 2:-2, 2:-2]

        masks_cut[masks_cut==2] = 0

        imgs_99 = imgs_cut.copy()

        # as HLA-ABC has a lot of outliers we will set a lower upper threshold to make sure that the "true" signals are not squeezed
        if m == "147Sm-HLA-ABC_Sm147":
            imgs_99[imgs_99 > np.percentile(imgs_99, 99)] = np.percentile(imgs_99, 99)
        else:
            imgs_99[imgs_99 > np.percentile(imgs_99, 99.5)] = np.percentile(imgs_99, 99.5)

        imgs_wo_bg = imgs_99 * masks_cut

        scaler = MinMaxScaler()
        tmp = imgs_wo_bg.reshape(-1, 1)
        tmp_norm = scaler.fit_transform(tmp)
        imgs_norm = tmp_norm.reshape(imgs_wo_bg.shape)

        img_df.loc[m] = [i for i in imgs_norm]
        mask_df.loc[m] = [i for i in masks_cut]

    stack_save(img_df, os.path.join(BM_path, s, "IF_IMC_processed"))
    stack_save(mask_df, os.path.join(BM_path, s, "IF_IMC_ilastik-masks"))

    print("Sample " + s + " finished and saved.")


In [16]:
from skimage.morphology import square, dilation
import numpy as np
bright_pixel = np.zeros((11,11))
bright_pixel[5,5] = 1
#dilation(bright_pixel, square(3))
dilation(bright_pixel, square(5))



array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])